<img 
    style="position: absolute; 
           left: 60%; 
           top: 0; /* Added to ensure proper positioning */
           height: 900px; 
           width: 40%; /* Maintain the original width */
           object-fit: cover; /* Adjust if necessary */
           clip-path: inset(0px 50px 0px 50px round 10px);" 
    src= "public_html\_data\fig\buildingdamage.svg"
/>
</figure>


<h1 style="width: 60%; color: #EC6842; font-size: 55pt;">
    <Strong>
        IJsselsteinseweg 77
    </Strong>
</h1>

<h2 id="Background"><B>
    Rationale for the project<a class="anchor-link" href="#Background">&#182;</a>
    </B>
</h2>
<p style="text-align: justify; width: 60%; font-weight: normal;">
     This studies case study is the building IJsselsteinseweg 77 in Nieuwegein in the Netherlands, a deatched 2-layer unreinforced masonry structure from 1961 with 2 floors and a cellar which was eventually demolished circa 2015. We can evaluate the state of damage before its demolishing due to a Foundation research report that was commissioned by the owner to Sandra Lobbe Ispectie & Advies, a building consulting firm who carried out the building and foundation investigation on June 2019 and thereafter produced a report documenting the buildings measurements, characteristics, damage state and their own assessment on the necessary course of action or risk of the building integrity due to subsidence effects. The main assessment found that the current capacity provided by the building’s foundation was Insufficient and given the experienced deformations by the building and the damage state of the building, countermeasures should be implemented in a period of 1-5 years following the assessment date. 
</p>

## <strong> X | Imports</strong>

In [1]:
from bricks import house, LTSM, EM
from bricks.tools.plots import EM_plot, LTSM_plot, subsurface

import numpy as np

## <strong> 0 | Instantiate `HOUSE` object</strong>

### <strong> 0.1 | Structure dimensions and measurements</strong>

In [2]:
walls = {
    'Wall 1':{"x": np.array([0, 0, 0]), "y": np.array([0, 3.5, 7]), "z": np.array([0, -72, -152]), 'phi': np.array([1/200,1/200]), 'height': 5250, 'thickness': 27,'area': 390600},
    'Wall 2':{"x": np.array([0, 4.5, 8.9]), "y": np.array([7, 7, 7]), "z": np.array([-152, -163, -188]),  'phi': np.array([1/33,1/50]), 'height': 5250, 'thickness': 27,'area': 464625},
    'Wall 3':{"x": np.array([8.9, 8.9]), "y": np.array([3.6, 7]), "z": np.array([-149, -188]), 'phi': np.array([0,0]), 'height': 5250, 'thickness': 27,'area': 166950},
    'Wall 4':{"x": np.array([8.9, 10.8]), "y": np.array([3.6, 3.6]), "z": np.array([-149,-138]), 'phi': np.array([0,0]), 'height': 2850, 'thickness': 27,'area': 49020},
    'Wall 5':{"x": np.array([10.8, 10.8]), "y": np.array([0, 3.6]), "z": np.array([-104, -138]), 'phi': np.array([1/77,1/67]), 'height': 2850, 'thickness': 27,'area': 96720},
    'Wall 6':{"x": np.array([0, 5.2, 6.4, 8.9, 10.8]), "y": np.array([0, 0, 0, 0, 0]), "z": np.array([0, -42, -55, -75, -104]), 'phi': np.array([1/100,1/100]), 'height': 5000, 'thickness': 27, 'area': 517980},
}

damage =  { 'crack_1' : {'wall_id' : 'Wall 2','c_w': 4,'c_l': 890,},
            'crack_2' : {'wall_id' : 'Wall 1','c_w': 2,'c_l': 1670,},
            'crack_3' : {'wall_id' : 'Wall 2','c_w': 3,'c_l': 1868,},
            'crack_4' : {'wall_id' : 'Wall 7','c_w': 0.1,'c_l': 2000,},
            'crack_5' : {'wall_id' : 'Wall 2','c_w': 2,'c_l': 1760,},
            'crack_6' : {'wall_id' : 'Wall 2','c_w': 1,'c_l': 994,},
            'crack_7' : {'wall_id' : 'Wall 2','c_w': 0.1,'c_l': 500,},
            'crack_8' : {'wall_id' : 'Wall 2','c_w': 2,'c_l': 993,}}

ijsselsteinseweg = house(measurements = walls)
ijsselsteinseweg.state = damage

### <strong> 0.2 | Estimating the displacement surface & greenfield profile</strong>

In [3]:
ijsselsteinseweg.interpolate() # Interpolate subsidence shape
ijsselsteinseweg.fit_function(i_guess = 1, tolerance = 1e-2, step = 1) # Fit gaussian shapes to walls

params = ijsselsteinseweg.soil['house'].values()
app = subsurface(ijsselsteinseweg, *params)
app.run_server(port=8050, debug=False)

From the above data we can observe how all curves increasingly progress towards the maximum displacement but do not resemble a gaussian shape which we assume to be the shape that subsidence patterns take. As a consequence we require to try to extrapolate outside the data to conitnuate the subisdence shape and we make use of an optimisation algorithm in order to find the values of the different variables in the function.   

## <strong> 1 | Assessing the damage of the building</strong>

### <strong> 1.1 | Assesment through Empirical Methods </strong>

In [4]:
ijsselsteinseweg.SRI(tolerance= 0.01) #Compute the SRI parameters
report = EM(ijsselsteinseweg.soil['sri'])

app = EM_plot(report)
app.run_server(port=8051, debug=False)

### <strong> 1.2 | Assesment through the LTSM </strong>

Include further assumptions as conditions
2. TILT VERIFICATION??
3. Height in sagging zone NA assumed to be at t= H/2 where as for hogging we assume the NA to be at the lower edge therfore t=H 
2. IDEA: Define as function determine potential maximum strain values and calibrate E/G ratio

In [5]:
limit_line = -1
eg_rat =  2.6  # Calibrate
LTSM(ijsselsteinseweg, limit_line, eg_rat, methods = ['greenfield','measurements'])

app = LTSM_plot(ijsselsteinseweg)    
app.run_server(port=8052, debug=True, dev_tools_ui=False)

In [6]:
report1 = ijsselsteinseweg.assessment['ltsm']['greenfield']['report']
report2 = ijsselsteinseweg.assessment['ltsm']['measurements']['report']

app = EM_plot(report2)
app.run_server(port=8054, debug=False)

### <strong> Y | Proces save data </strong>


In [7]:
dicts = [ijsselsteinseweg.soil['sri'],
         ijsselsteinseweg.process['params'],
         ijsselsteinseweg.assessment['ltsm']['greenfield']['results'],
         ijsselsteinseweg.assessment['ltsm']['measurements']['results'],
         ijsselsteinseweg.soil['shape']]
names = ['sri','gf_params','LTSM-GF','LTSM-MS', 'MS-VAR']
ijsselsteinseweg.process_dfs(dicts, names)

In [8]:
ijsselsteinseweg.dataframes['MS-VAR']

,inflection_points,regions,region_lengths,region_axes,d_deflection_zone
0,[3.5],[1],[7.0],"[[0.0, 7.0]]",[4.0]
1,[4.5],[1],[8.9],"[[0.0, 8.9]]",[7.202247191011235]
2,[],[0],[3.4],"[[3.6, 7.0]]",[0]
3,[],[0],[1.9000000000000004],"[[8.9, 10.8]]",[0]
4,[],[0],[3.6],"[[0.0, 3.6]]",[0]
5,"[5.2, 6.4, 8.9]","[-1, 1, 1]","[6.4, 3.7, 4.4]","[[0.0, 6.4], [5.2, 8.9], [6.4, 10.8]]","[2.6875, 2.297297297297291, 7.8409090909090935]"


In [9]:
ijsselsteinseweg.dataframes['sri']

,Smax,dSmax,DefRat,dDef,omega,phi,beta
0,152,152,2.17e-02,4.00e+00,1.03e-02,2.29e-02,1.27e-04
1,188,36,4.04e-03,7.20e+00,1.24e-03,5.68e-03,3.89e-02
2,188,39,1.15e-02,0.00e+00,1.15e-02,1.15e-02,0.00e+00
3,149,11,5.79e-03,0.00e+00,5.79e-03,5.79e-03,0.00e+00
4,138,34,9.44e-03,0.00e+00,9.44e-03,9.44e-03,0.00e+00
5,104,104,9.63e-03,7.84e+00,3.89e-03,1.53e-02,0.00e+00


In [10]:
drop_1 = ['lh_s','dl_s','lh_h','dl_h']
drop_2 = ['e_bh', 'e_bs','e_sh','e_ss','lh_s','lh_h','dl_s','dl_h']

dr = ijsselsteinseweg.dataframes['LTSM-GF']
dr.drop(drop_1, axis = 1)

rr = ijsselsteinseweg.dataframes['LTSM-MS']
rr.drop(drop_2, axis=1)

,e_tot,e_bt,e_dt,e_h,l_s,dw_s,l_h,dw_h
0,1.68e-04,1.68e-04,0.00e+00,0.00e+00,0.00e+00,0.00e+00,7.00e+03,4.00e+00
1,1.52e-03,2.74e-03,1.52e-03,3.04e-03,0.00e+00,0.00e+00,8.90e+03,7.20e+00
2,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
3,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
4,7.67e-04,7.67e-04,3.84e-04,7.67e-04,0.00e+00,0.00e+00,0.00e+00,0.00e+00
5,3.51e-04,3.51e-04,0.00e+00,0.00e+00,6.40e+03,2.69e+00,4.40e+03,7.84e+00


In [11]:
rr

,e_tot,e_bt,e_dt,e_bh,e_bs,e_sh,e_ss,e_h,l_s,lh_s,dl_s,dw_s,l_h,lh_h,dl_h,dw_h
0,1.68e-04,1.68e-04,0.00e+00,1.68e-04,0.00e+00,6.91e-04,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,7.00e+03,1.33e+00,5.71e-04,4.00e+00
1,1.52e-03,2.74e-03,1.52e-03,2.96e-04,0.00e+00,9.59e-04,0.00e+00,3.04e-03,0.00e+00,0.00e+00,0.00e+00,0.00e+00,8.90e+03,1.70e+00,8.09e-04,7.20e+00
2,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
3,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
4,7.67e-04,7.67e-04,3.84e-04,0.00e+00,0.00e+00,0.00e+00,0.00e+00,7.67e-04,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
5,3.51e-04,3.51e-04,0.00e+00,3.51e-04,2.26e-04,2.20e-03,8.16e-05,0.00e+00,6.40e+03,2.56e+00,4.20e-04,2.69e+00,4.40e+03,8.80e-01,1.78e-03,7.84e+00


In [12]:
x = ijsselsteinseweg.house['Wall 1']['y']
z = ijsselsteinseweg.house['Wall 1']['z']

np.gradient(z,x)

array([-20.57142857, -21.71428571, -22.85714286])